https://chatgpt.com/share/692af108-8220-8006-9d87-5d8a20c25aa4


so this is the all ideas of fine tuning which might be help ful

In [1]:
# Install unsloth with CUDA support
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install compatible versions based on unsloth-zoo requirements
!pip install "transformers>=4.51.3,<4.57.2" "datasets>=3.4.1,<4.4.0" "trl>=0.18.2,<=0.24.0" --upgrade

# Install other required packages
!pip install accelerate bitsandbytes wandb --upgrade

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-6uwk1zgo/unsloth_32d6c11e354e4b9ead4c29b31fc9add2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-6uwk1zgo/unsloth_32d6c11e354e4b9ead4c29b31fc9add2
  Resolved https://github.com/unslothai/unsloth.git to commit bda9e3d39b425f902d29e80c1f2870be7048d9c3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 75.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of t

In [ ]:
# Step3: Import necessary libraries
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:

# Step4: Check HF token
from google.colab import userdata
hf_token = userdata.get('HF_ACCESS_TOKEN')
login(hf_token)

In [4]:
# Optional: Check GPU availability
# Test if CUDA is available
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU device: Tesla T4


In [ ]:
#!pip install --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo transformers timm

In [7]:
model_name = "meta-llama/Llama-3.1-8B"
max_sequence_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [8]:
dataset_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(dataset_name, split="train")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-9ad84bb9cf65a4(…):   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [10]:
dataset['text'][2]

'<s>[INST] Compose a professional email with the following points:\n\nMe chinese cook 10 years\nGood good cook\nPeople love my food\nCan you hire me? [/INST] Thanks for the information. Unfortunately, your initial message does not contain enough details or structure to compose a compelling professional email to a potential employer. Here are some suggestions for how we can improve the message:\n\n1) Let\'s first clarify the type of cooking position you are interested in. Are you looking for a job as a line cook, sous chef, executive chef, or something else? This will help determine how to emphasize relevant experience and skills.\n\n2) What specific experience do you have? For example, "Over 10 years of experience preparing authentic Sichuan and Cantonese cuisine." Highlight any relevant skills, like knife skills, cooking techniques, menu development, etc. Mention the types of cuisine you specialize in.\n\n3) Have you worked in a professional kitchen before? If so, mention the name and

In [11]:
from datasets import load_dataset
import re

SYSTEM_PROMPT = "You are a helpful assistant."

def convert_to_llama3(example):
    text = example["text"]

    # Extract user message inside [INST] ... [/INST]
    user_match = re.search(r"\[INST\](.*?)\[/INST\]", text, re.DOTALL)
    if not user_match:
        raise ValueError("Could not find user message in example:\n" + text)
    user_msg = user_match.group(1).strip()

    # Extract assistant message (after [/INST])
    assistant_part = text.split("[/INST]")[-1]
    assistant_msg = assistant_part.replace("</s>", "").strip()

    # Build LLAMA-3 chat string
    final_text = (
        "<|begin_of_text|>"
        "<|start_header_id|>system<|end_header_id|>\n"
        f"{SYSTEM_PROMPT}<|eot_id|>"
        "<|start_header_id|>user<|end_header_id|>\n"
        f"{user_msg}<|eot_id|>"
        "<|start_header_id|>assistant<|end_header_id|>\n"
        f"{assistant_msg}<|eot_id|>"
    )

    return {"text": final_text}


# Load dataset
dataset = load_dataset("mlabonne/guanaco-llama2-1k", split="train")

# Convert
dataset_llama3 = dataset.map(convert_to_llama3)

# Keep only "text" column
dataset_llama3 = dataset_llama3.remove_columns(
    [col for col in dataset_llama3.column_names if col != "text"]
)

dataset_llama3


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [12]:
dataset_llama3['text'][2]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\nCompose a professional email with the following points:\n\nMe chinese cook 10 years\nGood good cook\nPeople love my food\nCan you hire me?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\nThanks for the information. Unfortunately, your initial message does not contain enough details or structure to compose a compelling professional email to a potential employer. Here are some suggestions for how we can improve the message:\n\n1) Let\'s first clarify the type of cooking position you are interested in. Are you looking for a job as a line cook, sous chef, executive chef, or something else? This will help determine how to emphasize relevant experience and skills.\n\n2) What specific experience do you have? For example, "Over 10 years of experience preparing authentic Sichuan and Cantonese cuisine." Highlight any relevant skills, like knife ski

In [13]:
# Step9: Setup/Apply LoRA finetuning to the model

model_lora = FastLanguageModel.get_peft_model(
    model = model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3047,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2025.11.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [14]:
# Add this before creating the trainer
if hasattr(model, '_unwrapped_old_generate'):
    del model._unwrapped_old_generate

In [18]:
# SOLUTION 1: Explicit list handling with validation
def formatting_func(examples):
    """
    Ensures we always return a proper list of strings.
    """
    texts = examples['text']

    # Validate and ensure it's a list
    if not isinstance(texts, list):
        texts = [texts]

    # Ensure all items are strings
    texts = [str(text) for text in texts]

    return texts

# SOLUTION 2: If above doesn't work, try this alternative approach
# Remove dataset_text_field and use packing=False
def formatting_func_alt(examples):
    """
    Alternative formatting with explicit iteration.
    """
    output = []

    # Handle both single examples and batches
    if isinstance(examples['text'], list):
        # This is a batch
        for text in examples['text']:
            output.append(str(text))
    else:
        # Single example
        output.append(str(examples['text']))

    return output

# Now create the trainer with the formatting_func
trainer = SFTTrainer(
    model = model_lora,
    tokenizer = tokenizer,
    train_dataset = dataset_llama3,

    # Add the formatting function
    formatting_func = formatting_func,

    max_seq_length = max_sequence_length,
    dataset_num_proc = 1,

    # CRITICAL: Add packing parameter
    packing = False,

    # Define training args
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# Optional: Test your formatting function BEFORE creating trainer
print("=" * 60)
print("DEBUGGING FORMATTING FUNCTION")
print("=" * 60)

# Test with different batch sizes
for batch_size in [1, 2, 4]:
    print(f"\nTesting with batch_size={batch_size}")
    sample = dataset_llama3[:batch_size]
    result = formatting_func(sample)

    print(f"  Input type: {type(sample)}")
    print(f"  Input 'texts' type: {type(sample['text'])}")
    print(f"  Output type: {type(result)}")
    print(f"  Output length: {len(result)}")
    print(f"  First item type: {type(result[0])}")
    print(f"  First item length: {len(result[0])} chars")

    # Validate output
    assert isinstance(result, list), f"Output must be list, got {type(result)}"
    assert all(isinstance(item, str) for item in result), "All items must be strings"
    print("  ✅ Validation passed!")

print("\n" + "=" * 60)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

DEBUGGING FORMATTING FUNCTION

Testing with batch_size=1
  Input type: <class 'dict'>
  Input 'texts' type: <class 'list'>
  Output type: <class 'list'>
  Output length: 1
  First item type: <class 'str'>
  First item length: 1041 chars
  ✅ Validation passed!

Testing with batch_size=2
  Input type: <class 'dict'>
  Input 'texts' type: <class 'list'>
  Output type: <class 'list'>
  Output length: 2
  First item type: <class 'str'>
  First item length: 1041 chars
  ✅ Validation passed!

Testing with batch_size=4
  Input type: <class 'dict'>
  Input 'texts' type: <class 'list'>
  Output type: <class 'list'>
  Output length: 4
  First item type: <class 'str'>
  First item length: 1041 chars
  ✅ Validation passed!



In [21]:
# Setup WANDB
from google.colab import userdata
wnb_token = userdata.get("WANDB_API_TOKEN")
# Login to WnB
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-on-Medical-CoT-Dataset',
    job_type="training",
    anonymous="allow"
)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sahilkatiyar2024 (sahilkatiyar2024-mumbai-university-and-idol-news) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [22]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.138600
20,1.609700
30,1.624900
40,1.472400
50,1.347800
60,1.522900


train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▁▃▁▅
train/learning_rate,█▇▅▄▂▁
train/loss,█▃▃▂▁▃
total_flos,8619277613875200.0
train/epoch,0.48
train/global_step,60
train/grad_norm,0.63087
train/learning_rate,1e-05
train/loss,1.5229


In [24]:
dataset_llama3[2]

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\nCompose a professional email with the following points:\n\nMe chinese cook 10 years\nGood good cook\nPeople love my food\nCan you hire me?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\nThanks for the information. Unfortunately, your initial message does not contain enough details or structure to compose a compelling professional email to a potential employer. Here are some suggestions for how we can improve the message:\n\n1) Let\'s first clarify the type of cooking position you are interested in. Are you looking for a job as a line cook, sous chef, executive chef, or something else? This will help determine how to emphasize relevant experience and skills.\n\n2) What specific experience do you have? For example, "Over 10 years of experience preparing authentic Sichuan and Cantonese cuisine." Highlight any relevant skills, like 

In [25]:
Compose a professional email with the following points:\n\nMe chinese cook 10 years\nGood good cook\nPeople love my food\nCan you hire me?

Object `me` not found.


In [27]:
# Step10: Testing after fine-tuning
question = """Compose a professional email with the following points:\n\nMe chinese cook 10 years\nGood good cook\nPeople love my food\nCan you hire me?"""
FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer(question, return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response)

['<|begin_of_text|>Compose a professional email with the following points:\n\nMe chinese cook 10 years\nGood good cook\nPeople love my food\nCan you hire me? \n\nThis is a very short and simple email that is meant to get the attention of a potential employer. The main points of the email are that the writer has been cooking for 10 years, that they are good at cooking, and that people love their food. The email also asks if the employer would be interested in hiring them. This email is meant to be a short and simple introduction, and it is not meant to be a full job application.<|end_of_text|>']
